In [1]:
import csv as csv
import pandas as pd
import numpy as np
import folium
%load_ext autoreload
%autoreload 2
%matplotlib inline


In [2]:
hdffile = "E:/Transit_Casa/Output/sfmuni_monthly_ts.h5"
store = pd.HDFStore(hdffile)
df = store.get('stop_day')

In [21]:
after  = store.select('rs_tod', where="MONTH=Timestamp('2013-09-01') & DOW=1 & TOD='2200-0259' & ROUTE_SHORT_NAME='2' & DIR=0")  
after.to_csv('after.csv')

In [22]:
load = 0.0
for i, row in after.iterrows():
    load -= row['OFF']
    load += row['ON']
    after.at[i, 'LOAD_DEP'] = load

In [24]:
after.to_csv('after_2_2200-0259.csv')

In [5]:
# number of unique bus stops in san fran
len(df.STOP_ID.unique())

3744

In [3]:
df1 = df[df['MONTH'] == '2009-09-01']
df2 = df[df['MONTH'] == '2013-09-01']

In [5]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3563 entries, 10689 to 14251
Data columns (total 51 columns):
MONTH                 3563 non-null datetime64[ns]
DOW                   3563 non-null int64
AGENCY_ID             3563 non-null object
STOP_ID               3563 non-null int64
NUMDAYS               3563 non-null int64
OBSDAYS               3563 non-null int64
TRIP_STOPS            3563 non-null int64
OBS_TRIP_STOPS        3563 non-null int64
WGT_TRIP_STOPS        3538 non-null float64
HEADWAY_S             3545 non-null float64
FARE                  3563 non-null float64
STOPNAME              3563 non-null object
STOPNAME_AVL          3563 non-null object
STOP_LAT              3563 non-null float64
STOP_LON              3563 non-null float64
EOL                   3563 non-null int64
SOL                   3563 non-null int64
TIMEPOINT             3517 non-null float64
ARRIVAL_TIME_DEV      3517 non-null float64
DEPARTURE_TIME_DEV    3517 non-null float64
DWELL_S             

In [4]:
import sys 
sys.path.append('E:\Transit_Casa\Alex\Scripts')
import Muni_Estimates_day as muni


In [5]:
df_final = pd.merge(df1,df2, how = 'outer', on = ['DOW', 'AGENCY_ID', 'STOP_ID'], suffixes = ['_09','_13'])
df_final = df_final.reindex_axis(sorted(df_final.columns), axis=1)


In [ ]:
import 

In [9]:
import math
np.isnan(np.nan)

True

In [6]:
def choose_right_latlon(val1, val2): 
    if np.isnan(val1): 
        return val2
    else: 
        return val1

In [7]:
df_final['LAT'] = df_final.apply(lambda row: choose_right_latlon(row['STOP_LAT_09'], row['STOP_LAT_13']), axis=1)

In [8]:
df_final['LON'] = df_final.apply(lambda row: choose_right_latlon(row['STOP_LON_09'], row['STOP_LON_13']), axis=1)

,AGENCY_ID,ARRIVAL_TIME_DEV_09,ARRIVAL_TIME_DEV_13,BIKERACK_09,BIKERACK_13,CAPACITY_09,CAPACITY_13,CROWDED_09,CROWDED_13,CROWDHOURS_09,...,WGT_TRIP_STOPS_13,WHEELCHAIR_09,WHEELCHAIR_13,ON_DIFF,ON_P_DIFF(FLOAT),ON_P_DIFF,LAT,LON,ON_P_DIFF_FLOAT,STOPNAME
0,SFMTA,0.934607,1.757239,0.219167,1.592472,2877.954545,2560.95,0.060312,0.037341,21.520189,...,265.943718,0.000000,0.566667,-213.317617,-9.498760,-9.49876029208%,37.721190,-122.475096,-9.498760,19th Avenue & Holloway St
1,SFMTA,3.542134,NaN,0.399840,NaN,1093.909091,NaN,0.031747,NaN,4.584047,...,NaN,0.126824,NaN,NaN,NaN,nan%,37.765159,-122.477210,0.000000,19TH AVE & LINCOLN WAY
2,SFMTA,1.770704,-0.344797,0.000000,0.000000,1505.818182,1242.70,0.000000,0.000000,0.000000,...,101.784411,0.000000,0.000000,0.178389,8.394136,8.3941360784%,37.723914,-122.435387,8.394136,MISSION ST & OCEAN AVENUE
3,SFMTA,4.582892,2.994147,0.226340,0.133333,1437.882353,1562.40,0.000000,0.000000,0.000000,...,79.000000,0.000000,0.000000,-29.414361,-68.157279,-68.1572790684%,37.719192,-122.425802,-68.157279,DUBLIN ST & LAGRANDE AVE
4,SFMTA,2.883022,NaN,0.150224,NaN,804.681818,NaN,0.012185,NaN,0.929952,...,NaN,0.286364,NaN,NaN,NaN,nan%,37.788530,-122.396044,0.000000,1st St & Howard St


In [13]:
df_final.to_csv('E:\Transit_Casa\Alex\C_Drive/map.csv')

In [9]:
df_final['ON_DIFF'] = df_final['ON_13'] - df_final['ON_09']
df_final['ON_P_DIFF_FLOAT'] = ((df_final['ON_13'] - df_final['ON_09'])/df_final['ON_09'])*100
df_final['ON_P_DIFF'] = df_final.ON_P_DIFF_FLOAT.astype(str) + '%'

In [106]:
print(df_final.ON_13.head())
df_final.ON_09.head()

0    2032.424038
1            NaN
2       2.303550
3      13.742234
4            NaN
Name: ON_13, dtype: float64


0    2245.741655
1     426.308055
2       2.125161
3      43.156595
4      11.328142
Name: ON_09, dtype: float64

In [29]:
df_final.head()

,AGENCY_ID,ARRIVAL_TIME_DEV_09,ARRIVAL_TIME_DEV_13,BIKERACK_09,BIKERACK_13,CAPACITY_09,CAPACITY_13,CROWDED_09,CROWDED_13,CROWDHOURS_09,...,WAITHOURS_13,WGT_TRIP_STOPS_09,WGT_TRIP_STOPS_13,WHEELCHAIR_09,WHEELCHAIR_13,LAT,LON,ON_DIFF,ON_P_DIFF_FLOAT,ON_P_DIFF
0,SFMTA,0.934607,1.757239,0.219167,1.592472,2877.954545,2560.95,0.060312,0.037341,21.520189,...,393.842721,273.297197,265.943718,0.000000,0.566667,37.721190,-122.475096,-213.317617,-9.498760,-9.49876029208%
1,SFMTA,3.542134,NaN,0.399840,NaN,1093.909091,NaN,0.031747,NaN,4.584047,...,NaN,119.847627,NaN,0.126824,NaN,37.765159,-122.477210,NaN,0.000000,nan%
2,SFMTA,1.770704,-0.344797,0.000000,0.000000,1505.818182,1242.70,0.000000,0.000000,0.000000,...,0.228783,135.850975,101.784411,0.000000,0.000000,37.723914,-122.435387,0.178389,8.394136,8.3941360784%
3,SFMTA,4.582892,2.994147,0.226340,0.133333,1437.882353,1562.40,0.000000,0.000000,0.000000,...,2.838759,87.038251,79.000000,0.000000,0.000000,37.719192,-122.425802,-29.414361,-68.157279,-68.1572790684%
4,SFMTA,2.883022,NaN,0.150224,NaN,804.681818,NaN,0.012185,NaN,0.929952,...,NaN,63.000000,NaN,0.286364,NaN,37.788530,-122.396044,NaN,0.000000,nan%


In [28]:
min(df_final.ON_P_DIFF_FLOAT)

-100.0

In [17]:
df_final.columns

Index(['AGENCY_ID', 'ARRIVAL_TIME_DEV_09', 'ARRIVAL_TIME_DEV_13',
       'BIKERACK_09', 'BIKERACK_13', 'CAPACITY_09', 'CAPACITY_13',
       'CROWDED_09', 'CROWDED_13', 'CROWDHOURS_09',
       ...
       'WAITHOURS_13', 'WGT_TRIP_STOPS_09', 'WGT_TRIP_STOPS_13',
       'WHEELCHAIR_09', 'WHEELCHAIR_13', 'LAT', 'LON', 'ON_DIFF',
       'ON_P_DIFF_FLOAT', 'ON_P_DIFF'],
      dtype='object', length=104)

In [10]:
mapa = folium.Map([37.775, -122.4813333],
                  zoom_start=10,
                  tiles='cartodbpositron')


In [ ]:
def map(colmn_per,colmn_diff,df,show_list,outfile)
    mapa = folium.Map([37.775, -122.4813333],
                  zoom_start=10,
                  tiles='cartodbpositron')
                  
    for name, row in df.iterrows():
    if row['ON_P_DIFF_FLOAT'] == np.inf:
        popup_string =  show_list[0] + ': '+str(row['STOP_ID'])+ '\n' + '     Percent Diff: ' + str(row['ON_P_DIFF_FLOAT']) + '\n' + '     Difference: ' + str(row['ON_DIFF'])
        marker = folium.CircleMarker([row["LAT"], row["LON"]], 
                                     color='Blue',
                                     fill_color='Blue', 
                                     radius=5,
                                     fill_opacity = 0.3, popup=popup_string)
        marker.add_to(mapa)
    elif row['ON_P_DIFF'] == 'nan%':
        popup_string =  'Stop_ID: '+str(row['STOP_ID'])+ '\n' + '     Percent Diff: ' + str(row['ON_P_DIFF_FLOAT']) + '\n' + '     Difference: ' + str(row['ON_DIFF'])
        marker = folium.CircleMarker([row["LAT"], row["LON"]], 
                                     color='#6600cc',
                                     fill_color='#cc99ff', 
                                     radius=5,
                                     fill_opacity = 0.3, popup=popup_string)
        marker.add_to(mapa)

    else:
        popup_string =  'Stop_ID: '+str(row['STOP_ID'])+ '\n' + '     Percent Diff: ' + str(round(row['ON_P_DIFF_FLOAT'])) + '\n' + '     Difference: ' + str(round(row['ON_DIFF']))
        marker = folium.CircleMarker([row["LAT"], row["LON"]], 
                                     color=color(row['ON_P_DIFF_FLOAT'],df_final['ON_P_DIFF_FLOAT']), 
                                     fill_color=color(row['ON_P_DIFF_FLOAT'],df_final['ON_P_DIFF_FLOAT']), 
                                     radius=radius(row['ON_DIFF']),
                                     fill_opacity = 0.3, popup=popup_string)
        marker.add_to(mapa)
    mapa.save(outfile)


In [22]:
df_final.head()

,AGENCY_ID,ARRIVAL_TIME_DEV_09,ARRIVAL_TIME_DEV_13,BIKERACK_09,BIKERACK_13,CAPACITY_09,CAPACITY_13,CROWDED_09,CROWDED_13,CROWDHOURS_09,...,WAITHOURS_13,WGT_TRIP_STOPS_09,WGT_TRIP_STOPS_13,WHEELCHAIR_09,WHEELCHAIR_13,LAT,LON,ON_DIFF,ON_P_DIFF_FLOAT,ON_P_DIFF
0,SFMTA,0.934607,1.757239,0.219167,1.592472,2877.954545,2560.95,0.060312,0.037341,21.520189,...,393.842721,273.297197,265.943718,0.000000,0.566667,37.721190,-122.475096,-213.317617,-9.498760,-9.49876029208%
1,SFMTA,3.542134,NaN,0.399840,NaN,1093.909091,NaN,0.031747,NaN,4.584047,...,NaN,119.847627,NaN,0.126824,NaN,37.765159,-122.477210,NaN,NaN,nan%
2,SFMTA,1.770704,-0.344797,0.000000,0.000000,1505.818182,1242.70,0.000000,0.000000,0.000000,...,0.228783,135.850975,101.784411,0.000000,0.000000,37.723914,-122.435387,0.178389,8.394136,8.3941360784%
3,SFMTA,4.582892,2.994147,0.226340,0.133333,1437.882353,1562.40,0.000000,0.000000,0.000000,...,2.838759,87.038251,79.000000,0.000000,0.000000,37.719192,-122.425802,-29.414361,-68.157279,-68.1572790684%
4,SFMTA,2.883022,NaN,0.150224,NaN,804.681818,NaN,0.012185,NaN,0.929952,...,NaN,63.000000,NaN,0.286364,NaN,37.788530,-122.396044,NaN,NaN,nan%


In [27]:
int(min(df_final['ON_P_DIFF_FLOAT']))

-100

In [55]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3645 entries, 0 to 3644
Columns: 104 entries, AGENCY_ID to ON_P_DIFF
dtypes: datetime64[ns](2), float64(96), object(6)
memory usage: 2.9+ MB


In [24]:
df_final.ON_P_DIFF_FLOAT = df_final.ON_P_DIFF_FLOAT.replace(to_replace = np.nan, value = 0)

In [87]:
for c in df_final.columns:
    print(c)

AGENCY_ID
ARRIVAL_TIME_DEV_09
ARRIVAL_TIME_DEV_13
BIKERACK_09
BIKERACK_13
CAPACITY_09
CAPACITY_13
CROWDED_09
CROWDED_13
CROWDHOURS_09
CROWDHOURS_13
DEPARTURE_TIME_DEV_09
DEPARTURE_TIME_DEV_13
DOORCYCLES_09
DOORCYCLES_13
DOW
DWELL_09
DWELL_13
DWELL_S_09
DWELL_S_13
EOL_09
EOL_13
FARE_09
FARE_13
FULLFARE_REV_09
FULLFARE_REV_13
HEADWAY_S_09
HEADWAY_S_13
LOAD_ARR_09
LOAD_ARR_13
LOAD_DEP_09
LOAD_DEP_13
MONTH_09
MONTH_13
NUMDAYS_09
NUMDAYS_13
OBSDAYS_09
OBSDAYS_13
OBS_TRIP_STOPS_09
OBS_TRIP_STOPS_13
OFF_09
OFF_13
ONTIME5_09
ONTIME5_13
ON_09
ON_13
PASSDELAY_ARR_09
PASSDELAY_ARR_13
PASSDELAY_DEP_09
PASSDELAY_DEP_13
PASSHOURS_09
PASSHOURS_13
PASSMILES_09
PASSMILES_13
RDBRDNGS_09
RDBRDNGS_13
RUNSPEED_09
RUNSPEED_13
RUNSPEED_S_09
RUNSPEED_S_13
RUNTIME_09
RUNTIME_13
RUNTIME_S_09
RUNTIME_S_13
SERVMILES_09
SERVMILES_13
SERVMILES_S_09
SERVMILES_S_13
SOL_09
SOL_13
STOPNAME_09
STOPNAME_13
STOPNAME_AVL_09
STOPNAME_AVL_13
STOP_ID
STOP_LAT_09
STOP_LAT_13
STOP_LON_09
STOP_LON_13
TIMEPOINT_09
TIMEPOINT_13
TO

In [11]:
df_final['ON_DIFF'] = df_final['SERVMILES_13'] - df_final['SERVMILES_09']
df_final['ON_P_DIFF_FLOAT'] = ((df_final['SERVMILES_13'] - df_final['SERVMILES_09'])/df_final['SERVMILES_09'])*100
df_final['ON_P_DIFF'] = df_final.ON_P_DIFF_FLOAT.astype(str) + '%'
df_final.head()

,AGENCY_ID,ARRIVAL_TIME_DEV_09,ARRIVAL_TIME_DEV_13,BIKERACK_09,BIKERACK_13,CAPACITY_09,CAPACITY_13,CROWDED_09,CROWDED_13,CROWDHOURS_09,...,WAITHOURS_13,WGT_TRIP_STOPS_09,WGT_TRIP_STOPS_13,WHEELCHAIR_09,WHEELCHAIR_13,LAT,LON,ON_DIFF,ON_P_DIFF_FLOAT,ON_P_DIFF
0,SFMTA,0.934607,1.757239,0.219167,1.592472,2877.954545,2560.95,0.060312,0.037341,21.520189,...,393.842721,273.297197,265.943718,0.000000,0.566667,37.721190,-122.475096,27.399224,98.824807,98.8248072702%
1,SFMTA,3.542134,NaN,0.399840,NaN,1093.909091,NaN,0.031747,NaN,4.584047,...,NaN,119.847627,NaN,0.126824,NaN,37.765159,-122.477210,NaN,NaN,nan%
2,SFMTA,1.770704,-0.344797,0.000000,0.000000,1505.818182,1242.70,0.000000,0.000000,0.000000,...,0.228783,135.850975,101.784411,0.000000,0.000000,37.723914,-122.435387,-0.953864,-25.076422,-25.0764220521%
3,SFMTA,4.582892,2.994147,0.226340,0.133333,1437.882353,1562.40,0.000000,0.000000,0.000000,...,2.838759,87.038251,79.000000,0.000000,0.000000,37.719192,-122.425802,-0.405600,-8.637873,-8.6378729809%
4,SFMTA,2.883022,NaN,0.150224,NaN,804.681818,NaN,0.012185,NaN,0.929952,...,NaN,63.000000,NaN,0.286364,NaN,37.788530,-122.396044,NaN,NaN,nan%


In [47]:
def color(col_value,colname):
    #minimum=int(min(colname))
    #maximum=int(max(colname))
    #set to -200 to encompase all valid points  
    #in the future need to find a way to set a minimum value for all data sets
    #need to make the percent difference change the color 
   
    if int(col_value) in range (-10000, -500):
        col='#b30000'
    elif int(col_value) in range (-500, -100):
        col='#e60000'
    elif int(col_value) in range(-100,-75):
        col='#ff1a1a'
    elif int(col_value) in range(-75,-50):
        col='#ff4d4d'
    elif int(col_value) in range(-50,-25):
        col='#ff8080'
    elif int(col_value) in range(-25,-10):
        col='#ffb3b3'
    elif int(col_value) in range(-10,10):
        col='#ffff99'
    elif int(col_value) in range(10,25):
        col= '#99ffbb'
    elif int(col_value) in range(25,50):
        col='#66ff99'
    elif int(col_value) in range(50,75):
        col='#33ff77'
    elif int(col_value) in range(75,100):
        col='#00ff55'
    elif int(col_value) in range(100,500):
        col='#00cc44'
    elif int(col_value) in range(500,10000):
        col='#009933'
    else:
        col='Fuchsia'
    return col

In [59]:
def radius(colmn_value):
    #minimum=int(min(colname))
    #maximum=int(max(colname))
    
    if abs(int(colmn_value)) in range(0,50):
        rad = 5
    elif abs(int(colmn_value)) in range(50,100):
        rad = 15
    elif abs(int(colmn_value)) in range(100,500):
        rad = 40
    elif abs(int(colmn_value)) in range(500,1500):
        rad = 80
    elif abs(int(colmn_value)) in range(1500,3000):
        rad = 125
    else:
        rad=500
    return rad

In [33]:
d = (max(df_final['ON_DIFF']) - min(df_final['ON_DIFF']))/11
int(d)

425

In [45]:
int(min(df_final['ON_DIFF']))

-1910

In [36]:
max(df_final['ON_DIFF'])

2772.6941326272286

In [82]:
-2 in range(-3,-1)

True

In [64]:
-2 in range(-3,2)

True

In [31]:
-15 in range(-25,-10)

True

In [57]:
for c in df_final.columns: 
    print(c)

AGENCY_ID
ARRIVAL_TIME_DEV_09
ARRIVAL_TIME_DEV_13
BIKERACK_09
BIKERACK_13
CAPACITY_09
CAPACITY_13
CROWDED_09
CROWDED_13
CROWDHOURS_09
CROWDHOURS_13
DEPARTURE_TIME_DEV_09
DEPARTURE_TIME_DEV_13
DOORCYCLES_09
DOORCYCLES_13
DOW
DWELL_09
DWELL_13
DWELL_S_09
DWELL_S_13
EOL_09
EOL_13
FARE_09
FARE_13
FULLFARE_REV_09
FULLFARE_REV_13
HEADWAY_S_09
HEADWAY_S_13
LOAD_ARR_09
LOAD_ARR_13
LOAD_DEP_09
LOAD_DEP_13
MONTH_09
MONTH_13
NUMDAYS_09
NUMDAYS_13
OBSDAYS_09
OBSDAYS_13
OBS_TRIP_STOPS_09
OBS_TRIP_STOPS_13
OFF_09
OFF_13
ONTIME5_09
ONTIME5_13
ON_09
ON_13
PASSDELAY_ARR_09
PASSDELAY_ARR_13
PASSDELAY_DEP_09
PASSDELAY_DEP_13
PASSHOURS_09
PASSHOURS_13
PASSMILES_09
PASSMILES_13
RDBRDNGS_09
RDBRDNGS_13
RUNSPEED_09
RUNSPEED_13
RUNSPEED_S_09
RUNSPEED_S_13
RUNTIME_09
RUNTIME_13
RUNTIME_S_09
RUNTIME_S_13
SERVMILES_09
SERVMILES_13
SERVMILES_S_09
SERVMILES_S_13
SOL_09
SOL_13
STOPNAME_09
STOPNAME_13
STOPNAME_AVL_09
STOPNAME_AVL_13
STOP_ID
STOP_LAT_09
STOP_LAT_13
STOP_LON_09
STOP_LON_13
TIMEPOINT_09
TIMEPOINT_13
TO

In [98]:
df_final.ONTIME5_09.min()

0.0

In [99]:
-np.inf

-inf

In [101]:
def get_names(colmn_name):
    COLMN_NAME_BASE = colmn_name + '_09'
    COLMN_NAME_FUTURE = colmn_name +'_13'
    COLMN_PER_STR = colmn_name + '_P_DIFF_STR'
    COLMN_PER = colmn_name + '_P_DIFF'
    COLMN_DIFF = colmn_name + '_DIFF'
    return COLMN_NAME_BASE, COLMN_NAME_FUTURE, COLMN_PER_STR, COLMN_PER, COLMN_DIFF

In [105]:
l = get_names('ON')

In [108]:
l[0]

'ON_09'

In [21]:
df_final[df_final['STOP_ID'] == 5463] & df_final[df_final['ON_DIFF'] == np.nan]

,AGENCY_ID,ARRIVAL_TIME_DEV_09,ARRIVAL_TIME_DEV_13,BIKERACK_09,BIKERACK_13,CAPACITY_09,CAPACITY_13,CROWDED_09,CROWDED_13,CROWDHOURS_09,...,WAITHOURS_13,WGT_TRIP_STOPS_09,WGT_TRIP_STOPS_13,WHEELCHAIR_09,WHEELCHAIR_13,LAT,LON,ON_DIFF,ON_P_DIFF_FLOAT,ON_P_DIFF
2063,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
str(np.nan)

'nan'

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 177415 entries, 0 to 177414
Data columns (total 51 columns):
MONTH                 177415 non-null datetime64[ns]
DOW                   177415 non-null int64
AGENCY_ID             177415 non-null object
STOP_ID               177415 non-null int64
NUMDAYS               177415 non-null int64
OBSDAYS               177415 non-null int64
TRIP_STOPS            177415 non-null int64
OBS_TRIP_STOPS        177415 non-null int64
WGT_TRIP_STOPS        175913 non-null float64
HEADWAY_S             176957 non-null float64
FARE                  177415 non-null float64
STOPNAME              177415 non-null object
STOPNAME_AVL          177415 non-null object
STOP_LAT              177415 non-null float64
STOP_LON              177415 non-null float64
EOL                   177415 non-null int64
SOL                   177415 non-null int64
TIMEPOINT             174761 non-null float64
ARRIVAL_TIME_DEV      174761 non-null float64
DEPARTURE_TIME_DEV    17476